###Incremental Data Ingestion

In [0]:
%sql
CREATE VOLUME workspace.raw.bronze;
CREATE VOLUME workspace.raw.silver;
CREATE VOLUME workspace.raw.gold;

In [0]:
%sql
CREATE VOLUME workspace.bronze.bronzevolume;
    
CREATE VOLUME workspace.silver.silvervolume;

CREATE VOLUME workspace.gold.goldvolume;


In [0]:
%python
dbutils.widgets.text("src","")

In [0]:
%python
src_value = dbutils.widgets.get("src")
src_value

In [0]:
%python
df = spark.readStream.format("cloudFiles")\
          .option("cloudFiles.format","csv")\
          .option("cloudFiles.schemaLocation",f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoint")\
          .option("cloudFiles.schemaEvolutionMode","rescue")\
          .load(f"/Volumes/workspace/raw/rawvolume/rawdata/{src_value}/")

In [0]:
%python
df.writeStream.format("delta")\
        .outputMode("append")\
        .trigger(once=True)\
        .option("checkpointLocation",f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoint")\
        .option("path",f"/Volumes/workspace/bronze/bronzevolume/{src_value}/data")\
        .start()

In [0]:
SELECT * FROM delta.`/Volumes/workspace/bronze/bronzevolume/bookings/data`